# 放射線１,２　プログラミング課題

## 3. シンチレータ発光シミュレーション (発展課題1e)

ここまでで，NIST ASTARのデータから実際的なBraggカーブを得ることができました！

これを用いることで，実験で実際に観察したような現象を再現することが可能です．

ここでは，ZnSシンチレータの発光現象をシミュレーションしてみます．

### 条件の整理

Am-241線源の5mm角の面から4.5 MeVのα線が一様等方に射出すると想定し，
この面を $z=0$ の $xy$ 平面と仮定します．

$z = z_{0}~(z_{0}>0)$の位置にシンチレータがあるとします．

α線はシンチレータに入射すると，入射時点でもっている運動エネルギーを全てシンチレータに落とし，静止すると仮定します．

シンチレータ発光量はα線からシンチレータが受け取ったエネルギーに比例すると仮定します．

したがって，

* 線源の面内から等方的にα線が射出する様子
* α線がシンチレータに到達したときに残っている運動エネルギー

を計算できれば，シンチレーション現象を再現できそうです．

### Bragg曲線積分関数の準備

まず，前の課題までで得たBragg曲線を，積分の形で取り出せるようにしましょう．

初期運動エネルギー $T_{0}$ のときのα線のBragg曲線を $B(x;T_{0})$ としたとき，

$$
E(x;T_{0}) = \int_{x}^{x_{\rm max}}dx' B(x;T_{0})
$$

で定義される $E(x;T_{0})$ を作ります．この $E(x)$ は，

位置が初期位置 $x=0$ から $x$ まで進んだときにα線が持っている「残りの運動エネルギー」です．

まず，前の課題と同様にBragg曲線を`numpy.array`の形式で得ておきます．

ただし，初期エネルギー $T_{0}$ は今後も変更することがあるので，

$T_{0}$を指定したときに対応するBragg曲線を得るような関数`getBragg(T0)`を作っておきます．
関数そのものと，飛程に対応する最大の$x$の値も返すようにしておくと便利です．

※`python`では`return`文で複数の変数を返すことが可能です．

In [ ]:
# numpyライブラリを取り込み，以下npという名前で略記します
import numpy as np

# numpy配列をひとつ作り，dataという変数に格納します．arrayとは「配列」の意味です．
data = np.array( [] )

# 以下はおなじみのファイル入力
with open('alpha_air.txt') as f:
    for line in f:
        strs = line.split()
        
        # ここでdataにappend()関数を用いて要素を追加します．appendとは「追加する」という意味です．
        data = np.append( data, [ float( strs[0] ), float( strs[1] ) ] )
        

# numpy配列はこのままだと１次元の列なので，これをN行2列の2次元の列に整形します．
# 左辺のdataは，結果をdataという変数に再代入するという意味です．
data = data.reshape(int(len(data)/2),2 )


#----------------------------------------------------------------------------
def getBragg( T0 ):
    Emin = 0
    x = 0
    
    T = T0
    
    # エネルギーステップを初期エネルギーの1/1000に設定
    dE = T0/1000
    
    # Bragg曲線を表す配列
    bragg = np.array( [] )
    
    # 無限ループ: break条件が必ず必要！
    while True:
        
        # 現在の運動エネルギーに対応するdE/dxを求め，
        # その時のdxを計算する．
        # このとき，"dE/dx"のdxは密度で規格化されていることに注意して，
        # [cm]の次元を与えるように適切に換算する必要がある．
        dEdx = ...
        dx = ...
        
        # Bragg曲線データに追加
        bragg = np.append( bragg, [x, dEdx] )
        
        # 位置 x と運動エネルギー T を更新する．
        x += ...
        T -= ...
    
        # もし運動エネルギーがEminを下回ったら打ち切る
        if T < Emin:
            break
    
    # データの整形を忘れずに！
    bragg = bragg.reshape( int(len(bragg)/2), 2 )
    
    xmax = x
    
    return bragg, xmax

次に，得られたBragg曲線を上の積分の式に合うように，

数値積分によって$E(x;T_{0})$を得ます．$E(x;T_{0})$を計算する関数を以下のように定義しましょう．

※このとき，元のBragg曲線を再計算するのは無駄なので，使いまわしができるようにします．

 そのために，Bragg曲線を表現する`numpy.array`を関数を計算する際の引数の一つに指定します．
 
 与えるBragg曲線は初期運動エネルギー $T_{0}$ に対応するので，
 生成関数に $T_{0}$ を直接与える必要はなくなります（陰に与えられている）．
 
積分は微小区間`dx`に分割してを位置`x`から`xmax`まで足すことで「近似的」に行うことにします．

（これを区分求積法と言います．）

このため，積分区間の終わりである`xmax`を前もって知っておくと便利です．

このため，`getBragg()`で先回りして`xmax`も返すようにしておいたのでした．

In [ ]:
# getBragg()を用いてBragg曲線を取得
T0 = 4.5 # [MeV]
bragg, xmax = getBragg( T0 )

def getBraggIntegral( x, xmax, bragg ):
    
    # 微小区間 dx は [0, xmax] を十分細かく分割すれば良い．
    # ここでは全区間を 100 の微小区間に分割することにする．
    # 必要に応じて調整するべき．
    
    dx = xmax / 100
    
    # 積分の初期値はゼロ．
    integral = 0.0
    
    # 積分変数としてyを用いる．初期値はx
    y = x
        
    # y < xmaxの間ループする．
    while y < xmax:
        # 線形補間によって位置 x のときの dE/dxをBragg曲線から得る
        dEdx = numpy.interp( y, bragg[:,0], bragg[:,1])
        
        # dE = dEdx * dx.
        # ただし dEdxは [MeV cm^2 / g] の次元で与えられており，
        # dx は [cm] の次元であることに注意する．
        
        integral += ...
        y += ...
    
    return integral

# 結果の確認
for x in [0.5, 1.0, 2.0, 3.0]:
    print( 'integral at x={:.1f} cm = {:.3f} MeV'.format( x, getBraggIntegral(x, xmax, bragg) ) )

結果の確認で，
```
integral at x=0.5 cm = 3.987 MeV
integral at x=1.0 cm = 3.427 MeV
integral at x=2.0 cm = 2.062 MeV
integral at x=3.0 cm = 0.067 MeV
```
という結果が返ってくれば正しく実装できています（ただし，空気密度として`1.2e-3 [g/cm3]`を使います）．


### α線のシミュレーション

次に，$z=0$面内の5mm角の位置から一様等方に出るα線を計算機上に作ります．

ここで強力なツールが「乱数」で，ランダムに数値を生成することを何回も繰り返すことで，分布を近似的に再現します．

これをモンテカルロ・シミュレーションと言います．

モンテカルロ・シミュレーションが有効な問題は非常に幅広く，

厳密な解を得ることが難しい現実の様々な問題に対してアプローチできます．

今回のシミュレーションの場合は，
* α線の射出の初期 $(x,y)$ 位置：面内に一様分布
* α線の方向：等方
を満たせれば良いです．

`numpy`には乱数を生成するライブラリ`numpy.random`が用意されていますので，今回はこれを用います．

In [2]:
from numpy.random import *

例えば，10個の`[0,1]`区間の一様な乱数列`a`を得るには，以下のようにします．

In [3]:
a = rand(10)
print(a)

[0.45725801 0.99310701 0.20448272 0.43008322 0.54783842 0.61746546
 0.47815227 0.0182242  0.50310569 0.14726249]


また，2個の独立な変数についての乱数列の組10個は

In [7]:
a = rand(10,2)
print(a)

[[0.86382983 0.82747841]
 [0.27822592 0.62113304]
 [0.5392739  0.72784461]
 [0.57829084 0.89996737]
 [0.58662578 0.26017101]
 [0.53751873 0.5634072 ]
 [0.36560707 0.93284321]
 [0.76860246 0.85558857]
 [0.30626684 0.52573845]
 [0.86245085 0.50534177]]


のように作れます．

例えば，１つめの乱数を`x`方向の位置，２つめの乱数を`y`方向の位置に，それぞれ対応できます．

`[0,1]`の区間で与えられている一様乱数を，`[-2.5, 2.5]`に対応するようにうまく変換すれば，

5mm角の面内から一様に射出するα線をうまく表現できそうです！


### 等方分布のシミュレーション

次にα線が等方に出る様子を考えます．

方向を指定するには球座標

$$
x = r\sin\theta\cos\phi\\
y = r\sin\theta\sin\phi\\
z = r\cos\theta
$$

で $(\theta,\phi)$ の2つの角度を指定できればよいです．

このとき，方位角 $\phi$ が $[0,2\pi]$ の範囲を取り，

天頂角 $\theta$ は $[0,\pi]$ の範囲を取れば，球面を埋め尽くせることがわかります．

方位角 $\phi$ についてはどの方向も同じなので $[0,2\pi]$ の範囲に一様分布できれば良いので特に問題はなさそうです．

一方で，天頂角 $\theta$ については，そうなりません．

半径1の球の表面積が $4\pi$ になるようにするためには，

$$
4\pi = \int_{0}^{2\pi}d\phi\,\int_{0}^{\pi}\sin\theta d\theta
$$

のように積分するので，$\theta$については $\sin\theta$ の「重み」がかかっています．

この重みを正しくモンテカルロで表現できないと，正しい等方分布を得られません．

これを行うために，以下の変数の置換を行います：
$$
t = \cos\theta,
$$

$$
dt = -\sin\theta~d\theta,\\
\theta\in[0,\pi]\rightarrow t\in[1,-1]
$$

このように書くと

$$
\int_{0}^{\pi}\sin\theta d\theta = \int_{-1}^{1} dt
$$

となり，$t$に関しては一様分布で重みはないことが分かります．

つまり，乱数としては $t$ を得て，$\theta$ は $t$ の関数として $\theta = \arccos(t)$ として得ることにします．

なお，今回のシミュレーションでは上半球 $z>0$ 方向に出るα線だけ考えれば十分なので，

$t$ の取る範囲として $[0,1]$ を考えておけば十分です．



以上をまとめると，α線１つにつき $(x,y,\phi,t)$ に対応する4個の一様乱数を用意すれば良いことになります．

これらをまとめて，`N`個のα線の $(x,y,\theta,\phi)$ をランダムに生成する関数`alphaEvents()`を作りましょう．

In [9]:
# pythonで数学関数を扱うライブラリmathを取り込む
import math

def alphaEvents(N):
    # (x,y,phi,t)に対応する一様乱数の組をN個用意する．
    rands = rand(N,4)
    
    out = np.array( [] )
    # randsの書く変数を変換
    for r in rands:
        # x: [0,1] --> [-0.15, 0.15]. 1番目の乱数r[0]を用いる
        x = -0.25 + r[0] * 0.5
        
        # y: [0,1] --> [-0.15, 0.15]. 2番目の乱数r[1]を用いる
        y = ...
        
        # phi: [0,1] --> [0, 2pi]. 3番目の乱数r[2]を用いる
        # piを得るためには math.pi を用いる
        phi = ...
        
        # t: [0, 1] --> [0, 1]. 4番目の乱数r[3]を用いる
        t = r[3]
        
        # theta = arccos(t). math.acos()が使える
        theta = ...
        
        # 結果の変数は [x, y, theta, phi]
        out = np.append( out, [x, y, theta, phi] )
    
    return out

上の関数がうまく実装できていると，
```
[[-1.88025641e-01 -1.81754537e-01  1.27896146e+00  2.59962252e-01]
 [-1.23829548e-01 -4.26775693e-02  3.46944721e-01  5.97795422e+00]
 [-9.32784261e-02 -1.02548984e-01  6.09801813e-01  3.56629978e+00]
 [-1.18776364e-01 -8.27046656e-02  1.01851074e+00  1.68666510e+00]
 [-1.12439339e-01 -8.83872861e-03  6.50265914e-01  3.18124655e-01]
 [-2.05079102e-03 -1.65703631e-01  1.04222392e+00  3.80679117e+00]
 [-6.08818773e-02 -2.34117734e-01  1.29696823e+00  1.86086226e+00]
 [-7.26477779e-02 -1.55923417e-01  1.05941599e+00  1.47988049e+00]
 [-2.43437538e-02 -1.61633486e-01  8.46551659e-01  5.26803801e+00]
 [-2.62359755e-01 -1.46676088e-01  7.71986099e-01  1.93481547e+00]]
```
のような結果が返ってきます（数値は乱数のため試行ごとに異なります）．

各要素の値の範囲がそれぞれ $(x,y,\theta,\phi)$ の満たすべき範囲に入っていることを確認してください．


### シンチレータ面に当たる位置と運動エネルギーの計算

乱数を使って線源から射出するα線をいくらでも作ることができるようになりました．

次に，α線がシンチレータ面に当たる様子を計算します．

α線は多重散乱の影響を受けにくく，ほぼ直線的に進むという仮定を置けば，

幾何学的にシンチレータに当たる位置を計算できます．

いまそれぞれのα線について，初期位置$(x_{0},y_{0})$と方向$(\theta,\phi)$が与えられているので，
その飛跡は，飛行距離$r$を任意のパラメータとして，

$$
x = x_{0}+r\sin\theta\cos\phi\\
y = y_{0}+r\sin\theta\sin\phi\\
z = r\cos\theta\\
$$

で与えられます．この直線的な飛跡が $z=z_{0}$ を通過するときの $(x,y)$ が

α線が当たるシンチレータ面上の位置になります．

また，そのときのパラメータ $r$ は飛行距離です．つまり，

$$
r = z_{0}/\cos\theta\\
x = x_{0}+r\sin\theta\cos\phi\\
y = y_{0}+r\sin\theta\sin\phi\\
$$

が解になります．

ただし，飛行距離 $r$ がもし最大飛程 `rmax` より長ければ，α線はシンチレータに到達できません．

また，前に作っておいた関数`getBraggIntegral()`を用いることで，

シンチレータに落とす残りの運動エネルギーを計算できます．

`alphaEvents()`を用いて生成したα線それぞれのイベントについて，シンチレータの高さが`z0 [cm]`のときの，
* シンチレータに当たるかどうか(`True / False`)
* シンチレータに当たる位置`(x,y)` [cm]
* シンチレータに落とす運動エネルギー`E_deposit`[MeV]

を返す関数`scintillation( event, z0, rmax )`を作りましょう．

In [ ]:
def scintillation( event, z0, rmax ):
    # eventの第1要素はx0, 第2要素はy0, 第3要素はtheta, 第4要素はphi
    x0 = event[0] #[cm]
    y0 = event[1] #[cm]
    theta = event[2]
    phi = event[3]
    
    # パラメータの根を求める
    r = ...
    
    # シンチレータに当たるかどうかの計算
    isHit = ...
    
    # シンチレータに当たる位置の計算
    # 当たらない場合でも延長線上の点として一般的に計算できる
    x = ...
    y = ...
    
    # シンチレータに落ちるエネルギー E_deposit の計算
    # 上で求めている変数braggとxmaxを用いるので，globalを用いてそれを使うことを宣言しておく．
    global xmax
    global bragg

    E_deposit = ...
    
    return isHit, (x,y), E_deposit

あとは，各イベントについてこの関数を適用すれば，
シミュレレーションができます．

In [ ]:
import tqdm

N = 1000
z0 = ...
rmax = xmax

print( "z0 = ", z0, ", rmax = ", rmax)

events = alphaEvents(N)

In [ ]:
# eventsの各変数が期待通りの分布をしているか確認

from matplotlib import pyplot as plt

# x
plt.hist( events[:,0], range=[-0.5, 0.5], bins = 50 )
plt.show()

# y
plt.hist( events[:,1], range=[-0.5, 0.5], bins = 50 )
plt.show()

# theta
plt.hist( events[:,2], range=[0, math.pi/2], bins = 50 )
plt.show()

# phi
plt.hist( events[:,3], range=[0, 2.0*math.pi], bins = 50 )
plt.show()

In [ ]:
result = np.array( [] )

# tqdmを用いるとプログレスバーを表示してくれる！
for event in tqdm.tqdm(events):
    isHit, position, E_deposit = scintillation( event, z0, rmax )
    
    # シンチレータに当たったときだけ結果を保存する．
    if isHit:
        result = np.append( result, [position[0], position[1], E_deposit] )
        
result = result.reshape( int(len(result)/3), 3 )
print( result )

## 結果の図示

ここまでで，シンチレータに当たったα線の計算の骨子は終わりです．

最終的にシンチレータの発光の様子を，自分が肉眼で観察した結果と比べたいと思います．

上の実行例で`N`を十分大きなサンプル数（例：`N=10000`）などに取り，再実行します．

※`N`を非常に大きな数にするとかなり計算時間がかかります．

手早く確認するときは1000-10000程度のサンプル数で行い，最終結果を得る際に大きな数を実行すると良いです．

`numpy`には2次元ヒストグラムが用意されています．

In [ ]:
import matplotlib.cm as cm
from matplotlib.colors import LogNorm
import copy

# ヒストグラムの区切りを指定：np.linspace()は[min, max]を等間隔でN分割する関数．
xbins = np.linspace(-10, 10, 101)
ybins = np.linspace(-10, 10, 101)


# カラーマップ
cmap = copy.copy(plt.get_cmap('Blues_r'))
cmap.set_under('black')
cmap.set_bad((0,0,0))

# ヒストグラムに詰める際に各エントリに E_deposit の重みをかける．
h2 = plt.hist2d( x = result[:,0], y = result[:,1], range = [[-1, 1], [-1, 1]], bins = 70, 
            weights = result[:,2]/N, cmap=cmap, norm=LogNorm(vmin=1.e-4, vmax=1.e-1, clip=True) )
cbar = plt.colorbar()
cbar.set_label('Scintillation Brightness [a.u.]', rotation=270, fontsize=12, fontfamily='serif')
plt.title('z = {:.1f} cm'.format( z0 ), fontsize=18, fontfamily='serif' )
plt.xlabel('X [cm]', fontsize=18, fontfamily='serif')
plt.ylabel('Y [cm]', fontsize=18, fontfamily='serif')
plt.show()

様々な`z0`の値について結果の絵を作り，自分の観察の結果と比べて考察をし，提出してください．